# Задание 7.5
Скачайте набор данных о винах. Вам необходимо будет предсказать качество вина по его характеристикам.

Для начала подготовим данные к бинарной классификации. 

Условно разделим вино на хорошее и плохое. 

Хорошим вином будем называть вино, параметр quality которого 6, плохим — всё остальное.

In [82]:
import numpy as np # для матричных вычислений
import pandas as pd # для анализа и предобработки данных
import matplotlib.pyplot as plt # для визуализации
import seaborn as sns # для визуализации

from sklearn import linear_model # линейные модели
from sklearn import tree # деревья решений
from sklearn import ensemble # ансамбли
from sklearn import metrics # метрики

from sklearn.model_selection import train_test_split

from sklearn import set_config
set_config(transform_output='pandas')

%matplotlib inline
plt.style.use('seaborn-darkgrid')
%config InlineBackend.figure_format = 'retina'
plt.rcParams['figure.figsize'] = (10, 8)

In [83]:
data = pd.read_csv('data/winequality-red.zip', sep=';')
data

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5


In [84]:
TARGET_FEATURE = 'quality'

In [85]:
# Преобразую целевой признак
data['quality'] = np.where(data['quality'] >= 6, 1, 0)

In [86]:
data['quality'].value_counts(True) * 100

quality
1    53.470919
0    46.529081
Name: proportion, dtype: float64

In [87]:
# Создаем матрицу наблюдений X и столбец с ответами y
X = data.drop(columns=TARGET_FEATURE)
y = data[TARGET_FEATURE]

In [88]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [89]:
# Создаем объект класса случайный лес
rf = ensemble.RandomForestClassifier(
    n_estimators=500, # число деревьев
    criterion='gini', # критерий эффективности
    max_depth=8, # максимальная глубина дерева
    min_samples_leaf=10,
    max_features='sqrt', # число признаков из метода случайных подространств
    random_state=42 # генератор случайных чисел
)

# Обучаем модель 
rf.fit(X_train, y_train)

RandomForestClassifier(max_depth=8, min_samples_leaf=10, n_estimators=500,
                       random_state=42)

In [90]:
# Делаем предсказание класса
y_train_pred = rf.predict(X_train)
y_test_pred = rf.predict(X_test)

In [91]:
print('TRAIN accuracy:', round(metrics.accuracy_score(y_train, y_train_pred), 2))
print('TEST accuracy:', round(metrics.accuracy_score(y_test, y_test_pred), 2))

TRAIN accuracy: 0.85
TEST accuracy: 0.75


In [92]:
dt = tree.DecisionTreeClassifier(
    criterion='entropy', # критерий информативности
    max_depth=6, # максимальная глубина дерева
    random_state=42 # генератор случайных чисел
)

# Обучаем модель 
dt.fit(X_train, y_train)

DecisionTreeClassifier(criterion='entropy', max_depth=6, random_state=42)

In [93]:
# Делаем предсказание класса
y_train_pred = dt.predict(X_train)
y_test_pred = dt.predict(X_test)

In [94]:
print('TRAIN accuracy:', round(metrics.accuracy_score(y_train, y_train_pred), 2))
print('TEST accuracy:', round(metrics.accuracy_score(y_test, y_test_pred), 2))

TRAIN accuracy: 0.82
TEST accuracy: 0.73


Дерево решений чуть хуже, чем случайный лес